# 02: Create Multi-Agent System with Genie Agents

Create a supervisor-based multi-agent system using LangGraph and Databricks Genie agents.

**Pattern**: Based on [Databricks LangGraph Multi-Agent Genie Guide](https://docs.databricks.com/aws/en/generative-ai/agent-framework/multi-agent-genie)

**Key Features**:
- Supervisor agent pattern coordinating multiple Genie agents
- GenieAgent for accessing Genie Spaces (no custom UC Functions needed)
- Automatic authentication via GenieAgent
- ResponsesAgent wrapper for MLflow compatibility
- Streaming support via predict_stream()

**Architecture**:
```
Supervisor Agent
  ├─ Customer Behavior Genie Agent (space: 01f0b7572b3a185d9f69cd89bc4c7579)
  └─ Inventory Management Genie Agent (space: 01f09cdef66116e5940de4b384623be9)
```

**Prerequisites**: None! GenieAgent handles authentication automatically.

## Install Dependencies

In [ ]:
%pip install --quiet --upgrade mlflow databricks-langchain langgraph langchain-core
dbutils.library.restartPython()

## Verify Genie Space Access

**IMPORTANT**: Before creating the agent, verify you have permissions to access the Genie spaces. see notebook test-genie-access

## Create Agent Module (agent.py)

Using `%%writefile` to create standalone agent file with GenieAgent supervisor pattern.

**Note**: Only run this after verifying Genie space access above!

In [ ]:
import os
try:
    os.remove('agent.py')
except Exception as e:
    print(f"Error removing 'agent.py': {e}")

In [ ]:
%%writefile agent.py

import sys
sys.path.append('../src')

from typing import TypedDict, Annotated, Sequence, Any, Optional
from langchain_core.messages import AnyMessage, BaseMessage, AIMessage, HumanMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from databricks_langchain import ChatDatabricks, GenieAgent
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import ResponsesAgentRequest, ResponsesAgentResponse, create_text_output_item

# Import modular configuration
from config.agent_config import AGENT_CONFIG, DOMAINS
from config.prompts import SYSTEM_PROMPT


# ============================================================================
# State Management
# ============================================================================

class AgentState(TypedDict):
    """Agent state with type annotations."""
    messages: Annotated[list[AnyMessage], add_messages]


# ============================================================================
# Create Genie Agents (Lazy Loading)
# ============================================================================

# Store Genie agents globally to avoid re-initialization
_genie_agents = None

def get_genie_agents():
    """Lazy-load GenieAgent instances for each domain."""
    global _genie_agents
    
    if _genie_agents is None:
        _genie_agents = {}
        
        # Customer Behavior Genie Agent
        _genie_agents["customer_behavior"] = GenieAgent(
            genie_space_id=DOMAINS["customer_behavior"].genie_space_id,
            genie_agent_name=DOMAINS["customer_behavior"].name,
            description=DOMAINS["customer_behavior"].description,
        )
        
        # Inventory Management Genie Agent
        _genie_agents["inventory"] = GenieAgent(
            genie_space_id=DOMAINS["inventory"].genie_space_id,
            genie_agent_name=DOMAINS["inventory"].name,
            description=DOMAINS["inventory"].description,
        )
    
    return _genie_agents


# ============================================================================
# Supervisor Agent
# ============================================================================

def create_supervisor_agent(llm):
    """Create supervisor agent that routes queries to Genie agents."""
    
    def supervisor_node(state: AgentState):
        """Supervisor decides which agent(s) to call."""
        # Lazy-load Genie agents here (not at import time)
        genie_agents = get_genie_agents()

        messages = state["messages"]
        last_message = messages[-1]

        # Simple keyword-based routing
        query = last_message.content.lower()

        responses = []

        # Route to customer behavior agent
        if any(keyword in query for keyword in ["cart", "abandon", "customer", "behavior", "segment", "purchase"]):
            try:
                cb_response = genie_agents["customer_behavior"].invoke({"messages": [last_message]})
                # Extract the response message properly
                if cb_response and "messages" in cb_response and len(cb_response["messages"]) > 0:
                    response_msg = cb_response["messages"][-1]
                    content = response_msg.content if hasattr(response_msg, 'content') else str(response_msg)
                    responses.append(f"[Customer Behavior]\n{content}")
            except Exception as e:
                responses.append(f"[Customer Behavior]\n[Error]: {str(e)}")

        # Route to inventory agent
        if any(keyword in query for keyword in ["inventory", "stock", "overstock", "stockout", "supply"]):
            try:
                inv_response = genie_agents["inventory"].invoke({"messages": [last_message]})
                # Extract the response message properly
                if inv_response and "messages" in inv_response and len(inv_response["messages"]) > 0:
                    response_msg = inv_response["messages"][-1]
                    content = response_msg.content if hasattr(response_msg, 'content') else str(response_msg)
                    responses.append(f"[Inventory]\n{content}")
            except Exception as e:
                responses.append(f"[Inventory]\n[Error]: {str(e)}")

        # If no specific routing, provide guidance
        if not responses:
            response_text = "I can help you with:\n- Customer behavior analysis (cart abandonment, segmentation, purchase patterns)\n- Inventory management (stock levels, stockouts, overstock)\n\nPlease ask a question related to these topics."
        else:
            response_text = "\n\n".join(responses)

        # Return properly formatted state update
        return {"messages": [AIMessage(content=response_text)]}
    
    # Build simple graph with supervisor node
    workflow = StateGraph(AgentState)
    workflow.add_node("supervisor", supervisor_node)
    workflow.set_entry_point("supervisor")
    workflow.add_edge("supervisor", END)
    
    return workflow.compile()


# ============================================================================
# ResponsesAgent Wrapper
# ============================================================================

class MultiGenieResponsesAgent(ResponsesAgent):
    """ResponsesAgent wrapper for multi-Genie agent system."""
    
    def __init__(self, agent_graph):
        self.agent = agent_graph
    
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        """Synchronous prediction."""
        import uuid
        
        # Convert request messages to LangChain format
        messages = []
        for msg in request.input:
            msg_dict = msg.model_dump() if hasattr(msg, 'model_dump') else msg
            role = msg_dict.get("role", "user")
            content = msg_dict.get("content", "")
            
            if role == "user":
                messages.append(HumanMessage(content=content))
            elif role == "assistant":
                messages.append(AIMessage(content=content))
        
        # Invoke agent with properly formatted state (just messages, no custom fields)
        result = self.agent.invoke({"messages": messages}, config={"configurable": {}})
        
        # Extract response
        final_message = result["messages"][-1]
        
        # Use helper method to create properly validated output with unique ID
        return ResponsesAgentResponse(
            output=[create_text_output_item(
                text=final_message.content,
                id=f"msg_{uuid.uuid4().hex[:8]}"
            )],
            custom_outputs=request.custom_inputs
        )
    
    def predict_stream(self, request: ResponsesAgentRequest):
        """Streaming prediction."""
        from mlflow.types.responses import output_to_responses_items_stream
        
        # Convert request messages to LangChain format
        messages = []
        for msg in request.input:
            msg_dict = msg.model_dump() if hasattr(msg, 'model_dump') else msg
            role = msg_dict.get("role", "user")
            content = msg_dict.get("content", "")
            
            if role == "user":
                messages.append(HumanMessage(content=content))
            elif role == "assistant":
                messages.append(AIMessage(content=content))
        
        # Stream agent execution
        for event in self.agent.stream({"messages": messages}, stream_mode=["updates", "messages"], config={"configurable": {}}):
            if event[0] == "updates":
                for node_data in event[1].values():
                    if len(node_data.get("messages", [])) > 0:
                        yield from output_to_responses_items_stream(node_data["messages"])


# ============================================================================
# Agent Initialization
# ============================================================================

def initialize_agent():
    """Initialize multi-agent system with modular configuration."""
    # Initialize LLM from config
    llm = ChatDatabricks(
        endpoint=AGENT_CONFIG["model_endpoint"],
        temperature=AGENT_CONFIG["temperature"],
        max_tokens=AGENT_CONFIG["max_tokens"]
    )
    
    # Create supervisor agent (Genie agents are lazy-loaded)
    agent_graph = create_supervisor_agent(llm)
    
    # Wrap in ResponsesAgent
    return MultiGenieResponsesAgent(agent_graph)


# Create agent instance for MLflow
AGENT = initialize_agent()

## Restart Python

After creating agent.py, restart Python to load the module.

**Note**: Run this cell, then proceed to test the agent.

In [ ]:
dbutils.library.restartPython()

## Test Agent - Single Domain Query

Test the agent with a simple inventory query.

In [ ]:
from agent import AGENT
from mlflow.types.responses import ResponsesAgentRequest

# Test query - should route to inventory agent
test_query = "What 5 products are at risk for overstock?"

print(f"Query: {test_query}\n")

# Create request with message dict
request = ResponsesAgentRequest(
    input=[{"role": "user", "content": test_query}]
)

# Test synchronous prediction
response = AGENT.predict(request)

print(f"Response:\n{response.output[0]}")
print(f"\n✅ Agent test PASSED")

## Test Agent - Multi-Domain Query with Streaming

Test the agent with a query that should route to both customer behavior and inventory agents.

In [ ]:
from mlflow.types.responses import ResponsesAgentRequest

# Test multi-domain query - should route to both agents
test_query = "What products are frequently abandoned in carts and do we have inventory issues with those items?"

# Create request
request = ResponsesAgentRequest(
    input=[{"role": "user", "content": test_query}]
)

print(f"Streaming response for: {test_query}\n")
print("Response: ")

for event in AGENT.predict_stream(request):
    if event.type == "response.output_item.done":
        print(event.item.get('text', ''), end="", flush=True)

print("\n\n✅ Streaming test PASSED")

## Log Agent to MLflow

Register the agent using `mlflow.models.set_model()` (Databricks pattern).

In [ ]:
import mlflow
import sys
sys.path.append('../src')

from mlflow.models.resources import DatabricksGenieSpace
from config.agent_config import AGENT_CONFIG, DOMAINS

# Set MLflow experiment
username = spark.sql("SELECT current_user()").collect()[0][0]
experiment_name = f"/Users/{username}/ml/experiments/multi-genie-agent"

mlflow.set_experiment(experiment_name)
print(f"MLflow experiment: {experiment_name}\n")

# Enable autolog for trace capture
mlflow.langchain.autolog()

# Prepare resources for automatic authentication passthrough
resources = []

# Add Genie spaces (GenieAgent handles authentication automatically)
for domain in DOMAINS.values():
    resources.append(DatabricksGenieSpace(genie_space_id=domain.genie_space_id))

print(f"Declaring {len(resources)} Genie Space resources for automatic auth passthrough:")
for resource in resources:
    print(f"  - {resource}")
print()

# Log agent
with mlflow.start_run(run_name="multi-genie-supervisor-agent") as run:
    # Log parameters
    mlflow.log_params({
        "model_endpoint": AGENT_CONFIG["model_endpoint"],
        "temperature": AGENT_CONFIG["temperature"],
        "max_tokens": AGENT_CONFIG["max_tokens"],
        "timeout_seconds": AGENT_CONFIG["timeout_seconds"],
        "agent_type": "multi_genie_supervisor"
    })
    
    # Log Genie space IDs as tags
    mlflow.set_tags({
        f"genie_space_{domain.name}": domain.genie_space_id
        for domain in DOMAINS.values()
    })
    
    # Import and log agent with automatic authentication passthrough
    from agent import AGENT
    
    # Create proper input example for ResponsesAgent
    input_example = {
        "input": [
            {"role": "user", "content": "What products are at risk for overstock?"}
        ]
    }
    
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=AGENT,
        resources=resources,
        input_example=input_example
    )
    
    run_id = run.info.run_id
    print(f"✅ Multi-agent system logged to MLflow with automatic authentication passthrough")
    print(f"Run ID: {run_id}")
    print(f"Model URI: {logged_agent_info.model_uri}")
    print(f"Experiment: {experiment_name}")

## Verify MLflow Registration

Check that the agent is properly registered and ready for deployment.

In [ ]:
# Get the current run
current_run = mlflow.get_run(run_id)

print("Registered Multi-Agent System Details:")
print(f"  Run ID: {current_run.info.run_id}")
print(f"  Status: {current_run.info.status}")
print(f"  Agent Type: {current_run.data.params.get('agent_type')}")
print(f"  Model Endpoint: {current_run.data.params.get('model_endpoint')}")
print(f"\n  Genie Spaces:")
for tag_key, tag_value in current_run.data.tags.items():
    if tag_key.startswith('genie_space_'):
        print(f"    - {tag_key}: {tag_value}")

print("\n✅ Multi-agent system ready for deployment")

## Register Model to Unity Catalog

Register the agent to Unity Catalog for production deployment and governance.

In [ ]:
# Set Unity Catalog as the model registry
mlflow.set_registry_uri("databricks-uc")

# Define UC model location
catalog = "juan_dev"
schema = "genai"
model_name = "retail_multi_genie_agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

print(f"Registering model to Unity Catalog: {UC_MODEL_NAME}")

# Get the model URI from the current run
model_uri = f"runs:/{run_id}/model"

# Register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=model_uri,
    name=UC_MODEL_NAME
)

print(f"\n✅ Model registered to Unity Catalog!")
print(f"   Model: {UC_MODEL_NAME}")
print(f"   Version: {uc_registered_model_info.version}")
print(f"   Run ID: {run_id}")

## Verify UC Registration

Confirm the model is available in Unity Catalog with all resources.

In [ ]:
# Load the registered model from UC
from mlflow import MlflowClient

client = MlflowClient()

# Get model version details
model_version_details = client.get_model_version(
    name=UC_MODEL_NAME,
    version=uc_registered_model_info.version
)

print("Unity Catalog Model Details:")
print(f"  Name: {model_version_details.name}")
print(f"  Version: {model_version_details.version}")
print(f"  Status: {model_version_details.status}")
print(f"  Source: {model_version_details.source}")

print("\n✅ Model successfully registered and ready for deployment!")

## Next Steps

✅ Multi-agent system created with GenieAgent supervisor pattern, logged to MLflow, and registered to Unity Catalog!

**Key Accomplishments:**
- GenieAgent for Genie Space access (no custom UC Functions!)
- Automatic authentication via GenieAgent (no WorkspaceClient issues)
- Supervisor pattern routes queries to appropriate domain agents
- Supports multi-domain queries across both Genie spaces
- ResponsesAgent wrapper for MLflow compatibility
- Streaming support via predict_stream()
- **Registered to Unity Catalog for production deployment**

**Unity Catalog Model**: `juan_dev.genai.retail_multi_genie_agent`

**Next**:
1. Open `03-test-agent.ipynb` to run comprehensive tests
2. Deploy model to Model Serving endpoint for production use